In [ ]:
import transformers, torch
from transformers import AutoModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("./Chinese-LLaMA-2-7B-hf", use_fast=False, trust_remote_code=True)

# Part2: The LLM generate Stage

- author: lyuchuny3@foxmail.com
- 知乎-link:https://zhuanlan.zhihu.com/p/1921601639858562902

## 1. LLaMA模型架构概览
LLaMA模型分为开头的embedding和后续的N个block (这个7B模型有32个blocks)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "./Chinese-LLaMA-2-7B-hf", 
    device_map="auto", 
    torch_dtype=torch.bfloat16, 
    trust_remote_code=True)

In [3]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(40076, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNo


## 2. generate生成流程概览

In [5]:
prompt="我喜欢攀岩"
inputs = tokenizer(prompt, return_tensors="pt")
x = inputs.input_ids #shape [1,7]
x,x.shape

(tensor([[    1, 29871, 30672, 31823, 33205, 33646, 31753]]),
 torch.Size([1, 7]))

以上面的输入为例，输入是input_ids的shape为[1,7]，则生成的流程是


1. [`embed_tokens`]： 将 token ID 映射为 embedding
2. [`layers 0-31`]: 逐层计算 hidden states。
3. [`norm`]: 最终归一化 hidden states
4. [`lm_head`]: 将 hidden states 映射为 logits
5. [`取最后一个位置的 logits`]:只关注序列最后一个位置的 logits
6. [`采样 token ID`]:根据 logits 采样下一个 token ID
7. [`循环`]: 将新 token ID 添加到序列中，重复步骤 1-6.直到满足停止条件，生成完整序列

### 1. Embedding 层（embed_tokens）​​
将输入的 token ID（整数）映射为对应的向量表示（embedding）。
```python3
输入 input_ids 的 shape 是 [1, 7]：（batch size = 1， seq_len =7)
输出 hidden_states 的 shape 是 [1, 7, 4096]：(embedding_dim =4096)
```
计算过程，embeding的权重是 weight [40076,4096]，则经过Gather [weight, inp]得到 output[1，7, 4096]

In [8]:
print("x.shape: ",x.shape)
hid_stat = model.model.embed_tokens(x) 
print("hid_state.shape: ", hid_stat.shape)

x.shape:  torch.Size([1, 7])
hid_state.shape:  torch.Size([1, 7, 4096])


### 2. Transformer Decoder 层（layers 0-31）​​
对 embed_tokens 输出的 hidden states 进行多层处理，逐步提取更高层次的语义信息。
```
输入和输出为hidden_states，shape 都为 [1, 7, 4096]
```
每一层 LlamaDecoderLayer 包含以下组件：Self-Attention，MLP, LayerNorm

In [9]:
 model.model.layers[0]

LlamaDecoderLayer(
  (self_attn): LlamaAttention(
    (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
    (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
    (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
    (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
    (rotary_emb): LlamaRotaryEmbedding()
  )
  (mlp): LlamaMLP(
    (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
    (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
    (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
    (act_fn): SiLUActivation()
  )
  (input_layernorm): LlamaRMSNorm()
  (post_attention_layernorm): LlamaRMSNorm()
)

In [10]:
hid_stat = model.model.layers[0](hid_stat)[0]      # [1, 7, 4096]
print(hid_stat.shape)

torch.Size([1, 7, 4096])


### 3. Final LayerNorm（norm）​​
对最后一层 LlamaDecoderLayer 输出的 hidden states 进行最终的归一化，确保数值稳定性。
```
输入和输出为hidden_states，shape 都为 [1, 7, 4096]
```

In [11]:
hid_stat = model.model.norm(hid_stat)              # [1, 7, 4096]
print(hid_stat.shape)

torch.Size([1, 7, 4096])


### 4. Language Model Head（lm_head）​​
lm_head 是一个线性层，其作用是将 hidden states 映射为 logits（词表上的得分），用于预测下一个 token 。
```
输入：hidden_states，shape 为 [1, 7, 4096]（来自 norm）。
输出：logits，shape 为 [1, 7, 40076]：
```
注意：logits 表示模型对当前输入序列中每个位置上所有可能的 token 的得分（未归一化的概率分布）。


In [12]:
hid_stat = model.lm_head(hid_stat)                 # [1, 7, 40076]
print(hid_stat.shape)

torch.Size([1, 7, 40076])


### 5. 取最后一个位置的 logits
​在自回归语言模型中，我们通常只关心序列中​​最后一个位置​​的 logits，因为模型是​​因果（causal）​​的：

模型在生成第 t 个 token 时，只能看到序列中第 0 到第 t-1 个 token 的信息。

因此，在采用之前，从logits [1, 7, 40076] 取最后一个位置的 logits，也就是logits [1, 40076]，这个作为采用的输入

In [15]:
logics = hid_stat[:,-1,:]
print("hid_stat.shape:",hid_stat.shape)
print("last logit shape:",logics.shape)

hid_stat.shape: torch.Size([1, 7, 40076])
last logit shape: torch.Size([1, 40076])


### 6. Sampling（采样）​
采样是为了从 logits 中采样下一个 token ID，作为模型预测的结果。
```
输入：logits [1, 40076] 
输出：1  # next token_id
```
根据 model.generate() 中的参数：

- do_sample=True,   启用采样策略（而不是直接取最大值）
- top_k=10,         只从 logits 中得分最高的 10 个 token 中进行采样
- top_p=0.85,       使用核采样（nucleus sampling），只从累计概率达到 85% 的 token 中进行采样
- temperature=1     控制采样的“随机性”，temperature=1 表示不调整概率分布。scores = scores / self.temperature

采样过程包括：

- 对 logits 应用 temperature（缩放 logits）。
- 可选：应用 top_k 和 top_p 过滤，限制采样的 token 范围。
- 通过 softmax 将 logits 转换为概率分布。
- 使用 torch.multinomial 从概率分布中采样一个 token ID。

采样得到的 token ID 是模型预测的下一个 token。

### 7. 自回归生成（Autoregressive Generation）​​
循环执行上述1-6步骤（从 embed_tokens 到采样），逐步生成完整的 token 序列。

- 初始输入 input_ids=[1, 7]
- 执行前向计算，采样得到第 8 个 token ID。
- 将新生成的 token ID 添加到输入序列中，形成新的输入（如 [1, 7]->[1,8] ）。
- 重复上述过程，直到满足停止条件（如达到 max_new_tokens 或遇到 eos_token_id）。

最终生成的 token ID 序列就是 generate_ids。


## 3. Prefill和Decoder过程
```
问题：prefill和decoder对应到这个生成过程的哪些执行过程呢？
```
Prefill 和 Decoder对应的是demo代码里的model.generate(）这一行过程。

对大模型推理进行**性能优化**的时候，会在生成阶段区分Prefill 和 Decoder：

​- ​Prefill（预填充）阶段​​
​- ​Decoder（解码 / 自回归生成）阶段​

对应到第二章节的生成过程，如果我的输入是7个tokens，输出是10个tokens，那么生成过程是

```python
prompt="我喜欢攀岩"
inputs = tokenizer(prompt, return_tensors="pt")
x = inputs.input_ids  #shape [1,7]

# prefill #######################################################
hid_stat = model.model.embed_tokens(x)             # [1, 7, 4096]
hid_stat = model.model.layers[0](hid_stat)[0]      # [1, 7, 4096]
...
hid_stat = model.model.layers[31](hid_stat)[0]     # [1, 7, 4096]
hid_stat = model.model.norm(hid_stat)              # [1, 7, 4096]
hid_stat = model.lm_head(hid_stat)                 # [1, 7, 40076]

logics = hid_stat[:,-1，：]    # get last token logis #[1, 40076]
next_token_id = sample(logics) #[1]

#decoder #########################################################
x_new # shape[1,1]
hid_stat_new = model.model.embed_tokens(x_new)      # [1, 1, 4096]
# ... layer 0-31, norm                              # [1, 1, 4096]
hid_stat_new = model.lm_head(hid_stat_new)          # [1, 1, 40076]
logits = hid_stat_new[:,-1，：]                       # [1, 1, 40076]
next_token_id = sample(logics) #[1]
```

1. ​​Prefill（预填充）阶段：​​
- 当传入 inputs.input_ids（即 prompt 对应的 token ID 序列）时，模型需要一次性计算出这个输入序列中​​所有 token 的 hidden states（隐藏状态）​​。
- 这个过程称为 ​​Prefill​​，因为它“预先填充”了输入部分的计算。
- 在这个阶段，模型会计算整个输入序列的 attention 和 hidden states，并缓存这些中间结果（通常称为 KV Cache），以便后续解码阶段复用。

2. ​​Decoder（解码 / 自回归生成）阶段：​​
- 在 Prefill 完成之后，模型开始​​逐个生成新的 token​​，也就是所谓的“自回归生成”。
- 每次生成一个新的 token，模型都会基于之前所有已生成的 token（包括 prompt 的 token 和已经生成的部分）来预测下一个 token。
- 这个过程是​​一步一步（token-by-token）​​进行的，直到达到 max_new_tokens 或遇到 eos_token_id（句子结束符）。